In [1]:
from bs4 import BeautifulSoup

In [2]:
raw_content = None

with open('fas_crse_cat.html', 'r') as f:
    raw_content = f.read()
    
soup = BeautifulSoup(raw_content, 'html.parser')

In [3]:
# all content will be in div.t
# so extract this
elts = soup.find_all('div', class_='t')

In [32]:
class Course(object):
    
    # static
    # this is the list of fields that are exported to CSV
    # see to_dict_for_csv()
    CSV_FIELDS = [
        'number',
        'semester',
        'name',
        'instructors',
        'schedule',
        'id',
    ]
    
    def __init__(self):
        self.number = None
        self.name = None
        self.instructors = []
        self.semester = None
        self.schedule = None
        self.strings = []
        self.id = None
        
    def __str__(self):
        return "{}".format(self.name)
        
    def __repr__(self):
        # for nicer ipython debugging
        return str(self.__dict__)
    
    def to_dict(self):
        # ALIAS
        return self.__dict__
#         return dict(
#             name=self.name,
#             instructors=self.instructors,
#             semester=self.semester,
#             schedule=self.schedule,
#             strings=self.strings
#         )

    def to_dict_for_csv(self):
        # returns a nicer-formatted dict ready for insertion into a csv
        # also everything needs to be converted to ascii
        
        def to_ascii(unicode_str):
            if unicode_str is None:
                return None
            return unicode_str.encode("ascii","replace")
        
        return dict(
            id=self.id,
            name=to_ascii(self.name),
            number=self.number,
            instructors=to_ascii((" & ".join(self.instructors))),
            semester=self.semester,
            schedule=self.schedule
        )
    
    


In [43]:
import re

# the .fs2 will be the title
# following .fs3 will be descriptive strings

courses = []
current_course = None

for elt in elts:    
    
    # first, ensure there's something actually here
    # FIXME could be bugs here from page breaks
    elt_text = None
    if elt.text is None or elt.text.strip() == "":
        continue
    else:
        elt_text = elt.text.strip()
    
    if set(['x5','fs2']).issubset(set(elt['class'])):
        # .x5.fs2 marks course numbers (e.g. COMPSCI 50)
        # and begins the start of a section for the course
        
        # start a new course
        current_course = Course()
        courses.append(current_course)
        
        current_course.number = elt_text
    elif set(['x5','hc','ff1','fs3']).issubset(set(elt['class'])):
        # this is course name (e.g. Introduction to Computer Science)
        # actually, it contains [course name, course id]
        # course id might not be in there, but don't worry about that for now
        course_name_list = list(elt)
        
        # if it doesn't have exactly 2 elements, it's malformed... skip!
        if len(course_name_list) != 2:
            continue
    
        # now unpack it
        [course_name, course_id_container] = list(elt)
        
        # write the name
        current_course.name = course_name.strip()
        
        # now the id
        # the container holds the id... but the id is also
        # within parentheses!
        # first trim whitespace...
        course_id = course_id_container.text.strip()
        
        if len(course_id) > 0:
            # as long as the course id isn't empty
            # (sometimes it is)

            # then remove parentheses
            course_id_without_parens = course_id[1:-1]
            
            current_course.id = course_id_without_parens

    elif set(['h10','ff1','fs5']).issubset(set(elt['class'])):
        # this is an alternative way for course ids to be placed
        # now, this includes BAD text like `FAS: Meets Foreign Lang Req Hausa`
        # as well as legit things which are like 
        # `(123591)`
        matcher = re.compile("\(\d+\)")
        results = matcher.findall(elt.text.strip())
        if len(results) > 0:
            # then we have a legit id number!!!!
            # remove parens
            # TODO: factor out to Course class
            current_course.id = elt.text.strip()[1:-1]
    elif set(['x5','ff3']).issubset(set(elt['class'])):
        # this is an instructor name!
        current_course.instructors.append(elt_text)
    elif elt is not None:
        # these are descriptive strings

        # error checking
        if current_course is None:
            continue
            
        # remove page footers
        BANNED_STRINGS = (
            'HARVARD UNIVERSITY',
            '9/4/2017 0:39 AM',
            '9/24/2017 0:39 AM',
            'HARVARD UNIVERSITY 9/24/2017 0:39 AM'
        )
        if elt_text in BANNED_STRINGS:
            continue
            
        current_course.strings.append(elt_text)

In [44]:
import re 

# Now try to pull meaningful info out of the strings
for course in courses:
    # try to extract a course schedule
    times = []
    for string in course.strings:
        matcher = re.compile("[MTWRF]{1,5} \d{4} [AP]M - \d{4} [AP]M")
        times.append(matcher.findall(string))
        
    flattened_times = sum(times,[])
    
    # this array will have max 1 element
    if len(flattened_times) > 0:
        course.schedule = flattened_times[0]
    else:
        course.schedule = None
    
    
    # try to extract a semester
    for string in course.strings:
        semester = re.search("20\d\d ((Fall)|(Spring))", string)
        if semester is not None:
            # just find the first one then quit
            course.semester = semester.group()
            break
        

In [45]:
courses[10]

{'name': u'Sophomore Tutorial: The Black Atlantic', 'schedule': u'R 0500 PM - 0659 PM', 'instructors': [u'Giovanna Micconi'], 'number': u'African & African Amer Studies   97', 'id': u'123590', 'semester': u'2018 Spring', 'strings': [u'2018 Spring (4 Credits)', u'Schedule:', u'R 0500 PM - 0659 PM', u'Instructor Permissions:', u'None', u'Enrollment Cap:', u'n/a', u'This course uses the concept of the "Black Atlantic" as a way to explore key theoretical issues in African, African', u'American, and Caribbean Studies. Interdisciplinary readings will examine the making and the construction of race', u'through different perspectives and experiences. The course is structured in three parts. Part I deals with Critical', u'Race Theory and the "Black Atlantic," seen as a conceptual space as well as a physical space of history formation.', u'Part II looks at Colonial and Postcolonial discourses on race and identity formation in diasporic and colonial', u'settings. Part III explores "Contemporary D

In [46]:
# dump data to file

# convert courses to dicts
course_dicts = [c.to_dict() for c in courses]

import json
with open('course_catalog_dump.json', 'w') as outfile:
    json.dump(course_dicts, outfile)

In [47]:
# write a csv
import csv

test_courses = courses[0:50]

with open('course_catalog_dump.csv', 'w') as csvfile:
    fieldnames = Course.CSV_FIELDS
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    
    for course in courses:
        writer.writerow(course.to_dict_for_csv())

In [48]:
courses[0]

{'name': u'Elementary Hausa', 'schedule': None, 'instructors': [u'John Mugane'], 'number': u'Hausa     AA', 'id': u'126307', 'semester': u'2017 Fall', 'strings': [u'2017 Fall (4 Credits)', u'Schedule:', u'TBD', u'Instructor Permissions:', u'None', u'Enrollment Cap:', u'n/a', u'A study of Hausa a most widely used native language and lingua franca in West Africa at the Elementary level (First', u'year part 1). Contact hours supplemented by digital resources. Emphasis on written expression, reading', u'comprehension, and oral fluency. Students must complete both terms of this course (parts AA and AB) within the', u'same academic year in order to receive full Elementary Hausa course credit.', u'Course Notes:', u'Languages in the tutorial program are offered when there is demonstrated', u'curricular or academic need on the part of the student, and when suitable', u'instruction can be arranged.  Please contact the department to learn more.', u'Additional Course Attributes:', u'Attribute Valu